In [1]:
import sys
from torchsummary import summary
import functools
import ast
import torch
import random
import numpy as np
import icecream as ic
import time
from IPython.display import clear_output
import matplotlib.pyplot as plt
sys.path.append("../../")
sys.path.append("../")
####### Dataset ############
from data_processor import DataProcessor
######## Search space #########
from search_space.RegNet import RegNet
from search_space.utils import create_widths_plot, scatter_results, get_generation_dfs
######## Training ###############
from trainer import Trainer, TrainerDistillation
from utils.train_cfg import get_cfg, show_cfg
###################################################
random_seed = 1
random.seed(random_seed)
# Set seed for NumPy
np.random.seed(random_seed)
# Set seed for PyTorch
torch.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)
# Additional steps if using CuDNN (optional, for GPU acceleration)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import os
from datetime import datetime
import itertools
import pandas as pd
import json
from io import StringIO
from coolname import generate_slug
from sklearn.metrics import accuracy_score
import torch.multiprocessing as mp


import torch.nn as nn
import types
import copy
import torch.nn.functional as F
from zcost_proxies.predictive import find_measures

/home/hpc/iwb3/iwb3021h/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_dataset_metadata(dataset_path):
    with open(os.path.join(dataset_path, 'metadata'), "r") as f:
        metadata = json.load(f)
        
    return metadata
# load dataset from file
def load_datasets(data_path, truncate):
    data_path = '../../datasets/'+data_path
    train_x = np.load(os.path.join(data_path,'train_x.npy'))
    train_y = np.load(os.path.join(data_path,'train_y.npy'))
    valid_x = np.load(os.path.join(data_path,'valid_x.npy'))
    valid_y = np.load(os.path.join(data_path,'valid_y.npy'))
    test_x = np.load(os.path.join(data_path,'test_x.npy'))
    metadata = load_dataset_metadata(data_path)

    if truncate:
        train_x = train_x[:64]
        train_y = train_y[:64]
        valid_x = valid_x[:64]
        valid_y = valid_y[:64]
        test_x = test_x[:64]

    return (train_x, train_y), \
           (valid_x, valid_y), \
           (test_x), metadata

def validation(model, val_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # 4. Calculate Accuracy
    accuracy = correct / total
    print('Accuracy on the test set: {:.2f}%'.format(accuracy * 100))

In [3]:
import json
import os

import kaleido
import pandas as pd
import numpy as np
import plotly.graph_objects as go
######## Search space #########
import sys
sys.path.append("../")
from search_space.RegNet import RegNet
from search_space.utils import create_widths_plot, scatter_results
# Initialize an empty list to store dictionaries
def results_to_df(path, name):
    data = []
    # Open the text file
    with open(path, 'r') as file:
        lines = file.readlines()
        # Initialize an empty dictionary to store data for each block
        block_data = {}
        for line in lines:
            # If the line contains dashes, it indicates the end of a block
            if '-------------------------' in line:
                # If block_data is not empty, add it to the list of data dictionaries
                if block_data:
                    data.append(block_data)
                    # Reset block_data for the next block
                    block_data = {}
            elif 'best_acc' in line:
                continue
            else:
                # Split the line by ':'
                #print(line)
                key, value = line.strip().split(': ')
                # Store the key-value pair in the block_data dictionary
                block_data[key] = value

    # Create a DataFrame from the list of dictionaries
    df = pd.DataFrame(data)

    # Convert columns to appropriate data types if needed
    df['epoch'] = df['epoch'].astype(int)
    df['lr'] = df['lr'].astype(float)
    df['train_acc'] = df['train_acc'].astype(float)
    df['train_loss'] = df['train_loss'].astype(float)
    df['test_acc'] = df['test_acc'].astype(float)
    df['test_acc_top5'] = df['test_acc_top5'].astype(float)
    df['test_loss'] = df['test_loss'].astype(float)
    df['epoch_time'] = df['epoch_time'].astype(float)
    df=df.assign(name=name)
    
    return df

metadata_dict={}
metadata_dict["Caitie"]={'num_classes': 4,
                         'input_shape': [49260, 3, 64, 64],
                         'codename': 'Caitie',
                         'benchmark': 47.008,
                         }
metadata_dict["Adaline"]={'num_classes': 20,
                         'input_shape': [50000, 3, 28, 28],
                         'codename': 'Adaline',
                         'benchmark': 89.85}
metadata_dict["Chester"]={'input_shape': [49998, 12, 8, 8],
                         'codename': 'Chester',
                         'benchmark': 57.826,
                         'num_classes': 3}
metadata_dict["Gutenberg"]={'input_shape': [45000, 1, 27, 18],
                         'codename': 'Gutenberg',
                         'benchmark': 40.98,
                         'num_classes': 6}
metadata_dict["LaMelo"]={'num_classes': 10,
                         'codename': 'LaMelo',
                         'input_shape': [50000, 1, 24, 24],
                         'benchmark': 85.2}
metadata_dict["Mateo"]={'num_classes': 10,
                         'input_shape': [50000, 3, 28, 28],
                         'codename': 'Mateo',
                         'benchmark': 90.87}
metadata_dict["Sadie"]={'input_shape': [50000, 3, 64, 64],
                         'codename': 'Sadie',
                         'benchmark': 80.33,
                         'num_classes': 10}
metadata_dict["Sokoto"]={'input_shape': [50000, 1, 9, 9],
                         'codename': 'Sokoto',
                         'benchmark': 0.0,
                         'num_classes': 9}
metadata_dict["CIFAR10"]={
                            "input_shape": [50000, 3, 32, 32],  
                            "codename": "CIFAR10",             
                            "benchmark": 90.65,                 
                            "num_classes": 10                  
                        }
def get_predictor_data_from_folder(base_folder,studies):

    
    train_data=[]
    for study_name in studies: 
        

        metadata=metadata_dict[study_name]
        rg=RegNet(metadata,
                        W0=[16, 120, 8],
                        WA=[16, 64, 8],
                        WM=[2.05,2.9,0.05],
                        D=[8,22,1], 
                        G=[8,8,8], 
                        base_config=f"../configs/search_space/config.yaml")
        data=[]
        study_folder=f"{base_folder}/{study_name}"
        subjects= os.listdir(study_folder)

        subjects=[subject for subject in subjects if os.path.isdir(f"{study_folder}/{subject}")]
        print(subjects)
        for name in subjects:
            try:
                test_acc=results_to_df(f"{study_folder}/{name}/worklog.txt", f"{name}")[["epoch","test_acc","name"]]
                #test_acc=test_acc.rename(columns={"test_acc":"acc","test_loss":"test_acc"})
                #print(test_acc.columns)
                if len(test_acc)>49:
                    test_acc_piv = test_acc[["epoch","test_acc","name"]].pivot(index='name', columns='epoch', values='test_acc').add_prefix("epoch_").reset_index()
                    test_acc_piv["best_acc"]=test_acc["test_acc"].max()
                    
                    model, info=rg.load_model(config_file=f"{study_folder}/{name}/config.yaml")
                    test_acc_piv["gen"]=1
                    test_acc_piv["num_stages"]=info["num_stages"]
                    test_acc_piv["params"]=info["params"]
                    test_acc_piv["WA"]=info["WA"]
                    test_acc_piv["W0"]=info["W0"]
                    test_acc_piv["WM"]=info["WM"]
                    test_acc_piv["DEPTH"]=info["DEPTH"]
                    test_acc_piv["num_classes"]=metadata["num_classes"]
                    test_acc_piv["num_channels"]=metadata["input_shape"][1]
                    test_acc_piv["benchmark"]=metadata["benchmark"]
                    
                    #print(info)
                    data.append(test_acc_piv)
            except:
                    pass

        data=pd.concat(data)
        train_data.append(data)
    train_data=pd.concat(train_data)
    return train_data
    

In [4]:
from torchvision.transforms import v2
class RandomPixelChange:
    def __init__(self, change_prob=0.1):
        self.change_prob = change_prob
    
    def __call__(self, img):
        # Convert image to numpy array
        img_array = np.array(img).astype(np.float32)
        
        # Normalize the array to [0, 1]
        img_array = (img_array - img_array.min()) / (img_array.max() - img_array.min())#
        
        unique_values=np.unique(img_array)
        
        # Generate a random mask with the same shape as img_array
        mask = np.random.rand(*img_array.shape) < self.change_prob
        
        # Apply the mask to randomly change the pixels to any of the unique values
        random_values = np.random.choice(unique_values, size=img_array.shape)
        img_array[mask] = random_values[mask]
        
        return img_array.transpose(1, 2, 0)



In [6]:
Dataset="Gutenberg"
(train_x, train_y), (valid_x, valid_y), (test_x), metadata = load_datasets(Dataset, truncate=False)
test_y = np.load(os.path.join('../../datasets/'+Dataset,'test_y.npy'))
metadata["select_augment"]=False
data_processor = DataProcessor(train_x[:], train_y[:], valid_x, valid_y, test_x, metadata)
train_loader, valid_loader, test_loader = data_processor.process()


metadata
dataset=metadata["codename"]

ic| f"selected transform {train_transform}": ('selected transform [<data_processor.RandomPixelChange object at '
                                              '0x7f97b3d05ee0>, ToTensor()]')
ic| self.x.shape: torch.Size([45000, 1, 27, 18])
ic| self.x.shape: torch.Size([15000, 1, 27, 18])
ic| self.x.shape: torch.Size([6000, 1, 27, 18])


In [7]:
rg=RegNet(metadata,
                    W0=[16, 120, 8],
                    WA=[16, 64, 8],
                    WM=[2.05,2.9,0.05],
                    D=[8,22,1], 
                    G=[8,8,8], 
                    base_config=f"../configs/search_space/config.yaml")

folder=f"/home/woody/iwb3/iwb3021h/NAS_COMPETITION_RESULTS/stages_pool"



In [11]:
np.unique(valid_loader.dataset.x[0])

array([0.14117648, 0.14509805, 0.14901961, 0.15294118, 0.15686275,
       0.16078432, 0.16470589, 0.16862746, 0.17254902, 0.1764706 ,
       0.18039216, 0.18431373, 0.1882353 , 0.19215687, 0.19607843,
       0.2       , 0.20392157, 0.20784314, 0.21176471, 0.21568628,
       0.21960784, 0.22352941, 0.22745098, 0.23137255, 0.23529412,
       0.23921569, 0.24313726, 0.24705882, 0.2509804 , 0.25490198,
       0.25882354, 0.2627451 , 0.26666668, 0.27058825, 0.27450982,
       0.2784314 , 0.28235295, 0.28627452, 0.2901961 , 0.29411766,
       0.29803923, 0.3019608 , 0.30588236, 0.30980393, 0.3137255 ,
       0.31764707, 0.32156864, 0.3254902 , 0.32941177, 0.33333334,
       0.3372549 , 0.34117648, 0.34509805, 0.34901962, 0.3529412 ,
       0.35686275, 0.36078432, 0.3647059 , 0.36862746, 0.37254903,
       0.3764706 , 0.38039216, 0.38431373, 0.3882353 , 0.39215687,
       0.39607844, 0.4       , 0.40392157, 0.40784314, 0.4117647 ,
       0.41568628, 0.41960785, 0.42352942, 0.42745098, 0.43137

In [8]:
unique_values=np.unique(valid_loader.dataset.x[0])
C,H,W=metadata['input_shape'][1:4]
PH,PW=int(H/8),int(W/8)
poss_augs= [

        [],
        [v2.RandAugment(magnitude=9)],
        [v2.RandAugment(magnitude=5)],
        [v2.RandAugment(magnitude=1)],
        [v2.TrivialAugmentWide(num_magnitude_bins=31)],
        [v2.TrivialAugmentWide(num_magnitude_bins=15)],
        [v2.AugMix(severity=3)],
        [v2.AugMix(severity=1)],
        ##########################
        [v2.RandomHorizontalFlip(),v2.RandomVerticalFlip()],
        [v2.RandomErasing(p=0.2, scale=(0.05, 0.2), ratio=(0.3, 3.3)), v2.RandomHorizontalFlip(),v2.RandomVerticalFlip()],
        [v2.RandomErasing(p=0.2, scale=(0.05, 0.2), ratio=(0.3, 3.3))],
        [v2.RandomErasing(p=0.2, scale=(0.02, 0.2), ratio=(0.3, 3.3)), v2.RandomCrop((H,W), padding=(PH,PW))],
        [v2.RandomCrop((H,W), padding=(PH,PW))],
        [v2.RandomCrop((H,W), padding=(PH,PW)), v2.RandomHorizontalFlip(),v2.RandomVerticalFlip()],
        [v2.RandomErasing(p=0.2, scale=(0.02, 0.2), ratio=(0.3, 3.3)),v2.RandomCrop((H,W), padding=(PH,PW)),v2.RandomHorizontalFlip()],
        ###########################################################
        [RandomPixelChange(0.01), v2.ToTensor()],
        [RandomPixelChange(0.025), v2.ToTensor()],
        [RandomPixelChange(0.05), v2.ToTensor()],
        [RandomPixelChange(0.01), v2.ToTensor(), v2.RandomHorizontalFlip(),v2.RandomVerticalFlip()],
        [RandomPixelChange(0.01), v2.ToTensor(),v2.RandomErasing(p=0.2, scale=(0.05, 0.2), ratio=(0.3, 3.3))],
        [RandomPixelChange(0.01), v2.ToTensor(), v2.RandomCrop((H,W), padding=(PH,PW))],
        [RandomPixelChange(0.01), v2.ToTensor(),v2.RandomHorizontalFlip(),v2.RandomVerticalFlip(), v2.RandomErasing(p=0.2, scale=(0.05, 0.2), ratio=(0.3, 3.3))]

    ]



In [9]:
train_data=get_predictor_data_from_folder(base_folder=folder,
                                          studies=[dataset])

['provocative_skua', 'passionate_hornet', 'rousing_cuckoo', 'modest_snail', 'glossy_fennec', 'versed_duck', 'giga_jackrabbit', 'amigurumi_camel', 'splendid_duck', 'amiable_smilodon', 'sweet_cat', 'handsome_eel', 'roaring_chameleon', 'meticulous_oyster', 'cunning_bandicoot', 'invaluable_leopard', 'organic_limpet', 'voracious_kagu', 'macho_seriema', 'clay_honeybee', 'jolly_spoonbill', 'loose_python', 'peach_marmot', 'graceful_flamingo', 'enthusiastic_binturong', 'xanthic_serval', 'spiritual_raptor', 'beige_limpet', 'dark_tuatara', 'therapeutic_frigatebird', 'prophetic_grouse', 'solid_chicken', 'tidy_silkworm', 'strong_finch', 'perfect_waxbill', 'crystal_walrus', 'pragmatic_roadrunner', 'aspiring_toad', 'muscular_poodle', 'fanatic_cheetah', 'nano_ibis', 'peridot_starling', 'maize_bulldog', 'stirring_seal', 'cherry_tapir', 'olivine_goat', 'fascinating_tarsier', 'married_bonobo', 'magic_deer', 'neat_termite', 'peridot_baboon', 'nice_leech', 'modest_hound', 'muscular_bullmastiff', 'refined_t

In [10]:
individuals=os.listdir(folder+"/"+dataset)
individuals=[ind for ind in individuals if os.path.isdir(os.path.join(folder+"/"+dataset, ind)) and ".ipynb" not in ind]
individuals = random.sample(individuals, 10)

In [11]:
dataset

'Gutenberg'

In [12]:
individuals

['voracious_kagu',
 'golden_scorpion',
 'prudent_crab',
 'delectable_octopus',
 'juicy_uakari',
 'splendid_duck',
 'tidy_silkworm',
 'invaluable_leopard',
 'amphibian_copperhead',
 'mega_frog']

In [13]:
preload_models=False
if preload_models:
    models={}
    for individual in individuals:
            ind_config=f"{folder}/{dataset}/{individual}/config.yaml"
            models[individual], _=rg.load_model(config_file=ind_config)
#models, chromosomes=rg.load_generation(folder+"/"+dataset)

In [88]:
from torch.utils.data import DataLoader, TensorDataset
from icecream import ic
import gc
##########################
train_loader.shuffle=False
tot_dfs=[]
for aug in range(len(poss_augs)):
    current_transforms=train_loader.dataset.transform.transforms
    train_loader.dataset.transform=v2.Compose(poss_augs[aug]+[current_transforms[-1]])
    ic("#############")
    ic(aug)
    ic(train_loader.dataset.transform.transforms)
    ###############
    


    #if "RandAugment"
    train_loader_iter = iter(train_loader)
    # Number of batches to extract
    num_batches =5

    # Initialize lists to hold the inputs and targets from the first 5 batches

    inputs_list = []
    targets_list = []
    try:
        for _ in range(num_batches):
            inputs, targets = next(train_loader_iter)  # Get the next batch
            inputs_list.append(inputs)
            targets_list.append(targets)
    except:
        continue
    # Concatenate the inputs and targets across the batches
    inputs = torch.cat(inputs_list)
    targets = torch.cat(targets_list)

    # Create a new TensorDataset from the selected data
    new_dataset = TensorDataset(inputs, targets)

    # Create a new DataLoader from this new dataset
    new_valid_loader = DataLoader(new_dataset, batch_size=train_loader.batch_size, shuffle=False)
    #train_loader.batch_size

    syn_scores={}
    measures=["fisher"]

    for individual in individuals:
                    ind_config=f"{folder}/{dataset}/{individual}/config.yaml"
                    model_aux, _=rg.load_model(config_file=ind_config)
                    #model_aux, _=regnet_space.create_model(params=params_dict[individual],save_folder=None, name=individual, gen=None, config_updates=None)
                    ic(len(new_valid_loader))
                    syn_scores[individual]=find_measures(model_aux.to("cuda"), new_valid_loader, ("random",len(new_valid_loader),metadata["num_classes"]), "cuda", F.cross_entropy, measures )

                    #syn_scores[individual]["params"]=str(params_dict[individuals])
                    del model_aux
                    gc.collect

    #########################################
    tot_df=pd.DataFrame(syn_scores).T.reset_index().rename(columns={"index":"name"})
    tot_df["aug"]=aug
    #tot_df["batch"]=batch
    tot_dfs.append(tot_df)
    #tot_df.to_csv(f"zcost_proxies/results/augs/{dataset}_t{aug}.csv")
tot_dfs=pd.concat(tot_dfs)



ic| '#############'
ic| aug: 0
ic| train_loader.dataset.transform.transforms: [Normalize(mean=tensor([0.0417]), std=tensor([0.1998]))]
ic| len(new_valid_loader): 5


Loading model: /home/woody/iwb3/iwb3021h/NAS_COMPETITION_RESULTS/stages_pool/LaMelo/cerulean_scorpion/config.yaml


/apps/jupyterhub/jh3.1.1-py3.11/envs/pytorch-2.2.0/lib/python3.12/site-packages/torch/nn/modules/module.py:1352: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
ic| len(new_valid_loader): 5


Loading model: /home/woody/iwb3/iwb3021h/NAS_COMPETITION_RESULTS/stages_pool/LaMelo/amiable_cow/config.yaml


ic| len(new_valid_loader): 5


...

Loading model: /home/woody/iwb3/iwb3021h/NAS_COMPETITION_RESULTS/stages_pool/LaMelo/fluffy_beaver/config.yaml


ic| len(new_valid_loader): 5


Loading model: /home/woody/iwb3/iwb3021h/NAS_COMPETITION_RESULTS/stages_pool/LaMelo/cyber_carp/config.yaml


ic| len(new_valid_loader): 5


Loading model: /home/woody/iwb3/iwb3021h/NAS_COMPETITION_RESULTS/stages_pool/LaMelo/ultraviolet_rattlesnake/config.yaml


In [14]:
from torch.utils.data import DataLoader, TensorDataset
from icecream import ic
import gc
##########################
train_loader.shuffle=False
tot_dfs=[]
for aug in range(len(poss_augs)):
    current_transforms=train_loader.dataset.transform.transforms
    train_loader.dataset.transform=v2.Compose(poss_augs[aug]+[current_transforms[-1]])
    ic("#############")
    ic(aug)
    ic(train_loader.dataset.transform.transforms)
    ###############
   
    #if "RandAugment"
    train_loader_iter = iter(train_loader)
    # Number of batches to extract
    num_batches =5

    # Initialize lists to hold the inputs and targets from the first 5 batches
    for batch in range(num_batches):
        inputs_list = []
        targets_list = []
        try:
            #for _ in range(num_batches):
            inputs, targets = next(train_loader_iter)  # Get the next batch
            inputs_list.append(inputs)
            targets_list.append(targets)
        except:
            continue
        # Concatenate the inputs and targets across the batches
        inputs = torch.cat(inputs_list)
        targets = torch.cat(targets_list)

        # Create a new TensorDataset from the selected data
        new_dataset = TensorDataset(inputs, targets)

        # Create a new DataLoader from this new dataset
        new_valid_loader = DataLoader(new_dataset, batch_size=train_loader.batch_size, shuffle=False)
        #train_loader.batch_size

        syn_scores={}
        measures=["fisher", "jacob_cov","plain"]

        for individual in individuals:
                        ind_config=f"{folder}/{dataset}/{individual}/config.yaml"
                        model_aux, _=rg.load_model(config_file=ind_config)
                        #model_aux, _=regnet_space.create_model(params=params_dict[individual],save_folder=None, name=individual, gen=None, config_updates=None)
                        ic(len(new_valid_loader))
                        syn_scores[individual]=find_measures(model_aux.to("cuda"), new_valid_loader, ("random",len(new_valid_loader),metadata["num_classes"]), "cuda", F.cross_entropy, measures )

                        #syn_scores[individual]["params"]=str(params_dict[individuals])
                        del model_aux
                        gc.collect

    #########################################
        tot_df=pd.DataFrame(syn_scores).T.reset_index().rename(columns={"index":"name"})
        tot_df["aug"]=aug
        tot_df["batch"]=batch
        tot_dfs.append(tot_df)
    #tot_df.to_csv(f"zcost_proxies/results/augs/{dataset}_t{aug}.csv")
tot_dfs=pd.concat(tot_dfs)


    #for metric in ["grasp","fisher","grad_norm","jacob_cov","plain","snip","synflow"]:
    #    if metric=="jacob_cov":
    #        df_syn[f"rank_{metric}"]=df_syn[metric].rank(ascending=False).astype(int)
    #    else:
    #        df_syn[f"rank_{metric}"]=df_syn[metric].rank().astype(int)

    #from scipy.stats import spearmanr
    #dict_spearman={}
    #dict_pearson={}
    #for metric in ["grasp","fisher","grad_norm","jacob_cov","plain","snip","synflow"]:
    ##for metric in ["snip"]:
    #        spearman_corr, _ = spearmanr(df_syn['best_acc'],df_syn[f"{metric}"])
    #        pearson_corr= df_syn['best_acc'].corr(df_syn[metric])

    #        dict_spearman[metric]=spearman_corr
    #        dict_pearson[metric]=pearson_corr

    #corr_df=pd.DataFrame([dict_spearman])
    #corr_df["dataset"]=dataset
    #df_syn.to_csv(f"zcost_proxies/results/augs/{dataset}_t{aug}.csv")

    #####################################################

ic| '#############'
ic| aug: 0
ic| train_loader.dataset.transform.transforms: [Normalize(mean=tensor([0.0370]), std=tensor([0.1889]))]
ic| len(new_valid_loader): 1


Loading model: /home/woody/iwb3/iwb3021h/NAS_COMPETITION_RESULTS/stages_pool/Gutenberg/voracious_kagu/config.yaml


/apps/jupyterhub/jh3.1.1-py3.11/envs/pytorch-2.2.0/lib/python3.12/site-packages/torch/nn/modules/module.py:1352: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
ic| len(new_valid_loader): 1


Loading model: /home/woody/iwb3/iwb3021h/NAS_COMPETITION_RESULTS/stages_pool/Gutenberg/golden_scorpion/config.yaml


ic| len(new_valid_loader): 1


...

Loading model: /home/woody/iwb3/iwb3021h/NAS_COMPETITION_RESULTS/stages_pool/Gutenberg/invaluable_leopard/config.yaml


ic| len(new_valid_loader): 1


Loading model: /home/woody/iwb3/iwb3021h/NAS_COMPETITION_RESULTS/stages_pool/Gutenberg/amphibian_copperhead/config.yaml


ic| len(new_valid_loader): 1


Loading model: /home/woody/iwb3/iwb3021h/NAS_COMPETITION_RESULTS/stages_pool/Gutenberg/mega_frog/config.yaml


In [97]:
from scipy.stats import pearsonr
pearsonr(tot_dfs_iter.groupby(["name","aug"]).mean()["fisher"].reset_index().sort_values(by=["name","aug"])["fisher"].values, tot_dfs.sort_values(by=["name","aug"])["fisher"].values)

PearsonRResult(statistic=0.9424422331242177, pvalue=3.356472883374027e-53)

In [18]:
subset_df

,name,fisher,jacob_cov,plain,aug,batch
0,intelligent_binturong,0.000000,0.000000,0.000000,0,0
0,intelligent_binturong,0.641970,0.005667,0.718140,1,0
0,intelligent_binturong,0.673172,0.006488,0.915502,2,0
0,intelligent_binturong,0.611319,0.006593,0.976939,3,0
0,intelligent_binturong,0.544628,0.002757,0.691178,4,0
0,intelligent_binturong,0.523205,0.007386,1.112402,5,0
0,intelligent_binturong,0.399015,0.005326,1.183349,6,0
0,intelligent_binturong,0.322286,0.002753,0.776693,7,0
0,intelligent_binturong,0.139632,0.005607,0.550229,8,0
0,intelligent_binturong,0.044861,0.003515,0.804435,9,0


In [27]:
from sklearn.preprocessing import MinMaxScaler
individuals=tot_dfs.name.unique()
num_batches=tot_dfs.batch.max()+1
rank_batches_fisher=[]
rank_batches_jac=[]
rank_batches_plain=[]
for individual in individuals:
    for batch in range(num_batches):
        subset_df=tot_dfs[(tot_dfs.name == individual) & (tot_dfs.batch == batch)]
        fisher_0 = subset_df.loc[subset_df['aug'] == 0, 'fisher'].values[0]
        jacob_cov_0 = subset_df.loc[subset_df['aug'] == 0, 'jacob_cov'].values[0]
        plain_0 = subset_df.loc[subset_df['aug'] == 0, 'plain'].values[0]

        # Divide the columns by the corresponding value where aug == 0
        subset_df['fisher'] = ((subset_df['fisher'] / fisher_0)-1).abs()
        subset_df['jacob_cov'] = ((subset_df['jacob_cov'] / jacob_cov_0)-1).abs()
        subset_df['plain'] = ((subset_df['plain'] / plain_0)-1).abs()
        
        subset_df=subset_df.set_index("aug")
        rank_batches_fisher.append(subset_df["fisher"]) 
        rank_batches_jac.append(subset_df["jacob_cov"]) 
        rank_batches_plain.append(subset_df["plain"]) 

rank_batches_fisher=pd.concat(rank_batches_fisher)
rank_batches_fisher=rank_batches_fisher.reset_index().groupby("aug").mean().sort_values(by="fisher")


rank_batches_jac=pd.concat(rank_batches_jac)
rank_batches_jac=rank_batches_jac.reset_index().groupby("aug").mean().sort_values(by="jacob_cov")

rank_batches_plain=pd.concat(rank_batches_plain)
rank_batches_plain=rank_batches_plain.reset_index().groupby("aug").mean().sort_values(by="plain")

/tmp/ipykernel_45604/76911826.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['fisher'] = ((subset_df['fisher'] / fisher_0)-1).abs()
/tmp/ipykernel_45604/76911826.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['jacob_cov'] = ((subset_df['jacob_cov'] / jacob_cov_0)-1).abs()
/tmp/ipykernel_45604/76911826.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

In [37]:
from sklearn.preprocessing import MinMaxScaler
individuals=tot_dfs.name.unique()
num_batches=tot_dfs.batch.max()+1
rank_batches_fisher=[]
for individual in individuals:
    for batch in range(num_batches):
        subset_df = tot_dfs[(tot_dfs.name == individual) & (tot_dfs.batch == batch)].sort_values(by="aug").set_index("aug")
        Q1 = subset_df["fisher"].quantile(0.1)
        Q3 = subset_df["fisher"].quantile(0.9)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        subset_df_no_outliers = subset_df[(subset_df["fisher"] >= lower_bound) & (subset_df["fisher"] <= upper_bound)]
        scaler = MinMaxScaler()
        subset_df_no_outliers["fisher"] = scaler.fit_transform(subset_df_no_outliers[["fisher"]])
        rank_batches_fisher.append(subset_df_no_outliers["fisher"])    

rank_batches_fisher=pd.concat(rank_batches_fisher)
rank_batches_fisher=rank_batches_fisher.reset_index().groupby("aug").mean().sort_values(by="fisher")

rank_batches_jac=[]
for individual in individuals:
    for batch in range(num_batches):
        subset_df = tot_dfs[(tot_dfs.name == individual) & (tot_dfs.batch == batch)].sort_values(by="aug").set_index("aug")
        Q1 = subset_df["jacob_cov"].quantile(0.1)
        Q3 = subset_df["jacob_cov"].quantile(0.9)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        subset_df_no_outliers = subset_df[(subset_df["jacob_cov"] >= lower_bound) & (subset_df["jacob_cov"] <= upper_bound)]
        scaler = MinMaxScaler()
        subset_df_no_outliers["jacob_cov"] = scaler.fit_transform(subset_df_no_outliers[["jacob_cov"]])
        rank_batches_jac.append(subset_df_no_outliers["jacob_cov"])      

rank_batches_jac=pd.concat(rank_batches_jac)
rank_batches_jac=rank_batches_jac.reset_index().groupby("aug").mean().sort_values(by="jacob_cov")


In [99]:
rank_batches_fisher.reset_index().groupby("aug").sum()["fisher"].sort_values()

aug
3     0.436968
2     0.473947
1     0.523396
4     0.705688
5     0.748343
7     1.386709
6     1.396586
14    2.992847
9     3.058030
8     3.067975
11    3.081058
21    3.087714
0     3.115826
19    3.163583
10    3.206098
20    3.305931
13    3.391763
15    3.411277
18    3.465862
12    3.510267
17    3.628700
16    3.732311
Name: fisher, dtype: float64

In [42]:
import ast
def convert_to_complex(value):
    try:
        # Try to convert the value to a complex number
        return ast.literal_eval(value)
    except (ValueError, SyntaxError):
        # Return the original value if it's not a valid complex string
        return value

# Apply the conversion function to all elements of the DataFrame
tot_dfs= tot_dfs.applymap(convert_to_complex)

/tmp/ipykernel_237340/340920632.py:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  tot_dfs= tot_dfs.applymap(convert_to_complex)


In [34]:
tot_dfs=pd.read_csv("zcost_proxies/results/augs/ranks_LaMelo.csv",index_col=0)

In [35]:

for col in ["fisher","jacob_cov"]:
    try:
        tot_dfs[col]=tot_dfs[col].astype(float)
    except:
        tot_dfs[col]=0

In [36]:
tot_dfs

,name,fisher,jacob_cov,aug,batch
0,devious_fossa,0.022094,-6.512279e+06,0,0
1,xanthic_ara,0.122732,-6.509233e+06,0,0
2,poised_dachshund,0.139703,-6.509555e+06,0,0
3,bronze_angelfish,0.002766,-6.507531e+06,0,0
4,russet_orca,0.261339,-6.509383e+06,0,0
...,...,...,...,...,...
5,gay_mink,0.036296,-6.507756e+06,21,0
6,sensible_dragon,0.091964,-6.509530e+06,21,0
7,enchanted_cockatoo,0.315380,-6.509949e+06,21,0
8,remarkable_jaguar,0.013051,-6.508843e+06,21,0


In [26]:
tot_dfs['jacob_cov'] = tot_dfs['jacob_cov'].apply(lambda x: abs(x))


In [27]:
tot_dfs.apply(lambda col: np.iscomplex(col).sum())

name         0
fisher       0
jacob_cov    0
plain        0
aug          0
batch        0
dtype: int64

In [28]:
tot_dfs["jacob_cov"]

0    4.799617e+06
1    4.799625e+06
2    4.799619e+06
3    4.799623e+06
4    4.799659e+06
         ...     
5    4.799639e+06
6    4.799609e+06
7    4.799621e+06
8    4.799623e+06
9    4.799610e+06
Name: jacob_cov, Length: 1100, dtype: float64

In [38]:
from sklearn.preprocessing import MinMaxScaler
individuals=tot_dfs.name.unique()
num_batches=tot_dfs.batch.max()+1
rank_batches_fisher=[]
for individual in individuals:
    for batch in range(num_batches):
        #rank_batches_fisher.append(tot_dfs[(tot_dfs.name==individual) & (tot_dfs.batch==batch)].sort_values(by="aug").set_index("aug")["fisher"].rank())
        subset_df = tot_dfs[(tot_dfs.name==individual) & (tot_dfs.batch==batch)].sort_values(by="aug").set_index("aug")
        scaler = MinMaxScaler()
        subset_df["fisher"] = scaler.fit_transform(subset_df[["fisher"]])
        rank_batches_fisher.append(subset_df["fisher"])

rank_batches_fisher=pd.concat(rank_batches_fisher)
rank_batches_fisher=rank_batches_fisher.reset_index().groupby("aug").mean().sort_values(by="fisher")

rank_batches_jac=[]
for individual in individuals:
    for batch in range(num_batches):
        #rank_batches_jac.append(tot_dfs[(tot_dfs.name==individual) & (tot_dfs.batch==batch)].sort_values(by="aug").set_index("aug")["jacob_cov"].rank())
        subset_df = tot_dfs[(tot_dfs.name==individual) & (tot_dfs.batch==batch)].sort_values(by="aug").set_index("aug")
        scaler = MinMaxScaler()
        subset_df["jacob_cov"] = scaler.fit_transform(subset_df[["jacob_cov"]])
        rank_batches_jac.append(subset_df["jacob_cov"])        
rank_batches_jac=pd.concat(rank_batches_jac)
rank_batches_jac=rank_batches_jac.reset_index().groupby("aug").mean().sort_values(by="jacob_cov")

rank_batches_plain=[]
for individual in individuals:
    for batch in range(num_batches):
        #rank_batches_plain.append(tot_dfs[(tot_dfs.name==individual) & (tot_dfs.batch==batch)].sort_values(by="aug").set_index("aug")["plain"].rank())
        subset_df = tot_dfs[(tot_dfs.name==individual) & (tot_dfs.batch==batch)].sort_values(by="aug").set_index("aug")
        scaler = MinMaxScaler()
        subset_df["plain"] = scaler.fit_transform(subset_df[["plain"]])
        rank_batches_plain.append(subset_df["plain"])
        
rank_batches_plain=pd.concat(rank_batches_plain)
rank_batches_plain=rank_batches_plain.reset_index().groupby("aug").mean().sort_values(by="plain", ascending=True)

rank_batches_syn=[]
for individual in individuals:
    for batch in range(num_batches):
        #rank_batches_syn.append(tot_dfs[(tot_dfs.name==individual) & (tot_dfs.batch==batch)].sort_values(by="aug").set_index("aug")["synflow"].rank())
        subset_df = tot_dfs[(tot_dfs.name==individual) & (tot_dfs.batch==batch)].sort_values(by="aug").set_index("aug")
        scaler = MinMaxScaler()
        subset_df["synflow"] = scaler.fit_transform(subset_df[["synflow"]])
        rank_batches_syn.append(subset_df["synflow"])
rank_batches_syn=pd.concat(rank_batches_syn)
rank_batches_syn=rank_batches_syn.reset_index().groupby("aug").mean().sort_values(by="synflow", ascending=True)

rank_batches_snip=[]
for individual in individuals:
    for batch in range(num_batches):
        #rank_batches_snip.append(tot_dfs[(tot_dfs.name==individual) & (tot_dfs.batch==batch)].sort_values(by="aug").set_index("aug")["snip"].rank())
        subset_df = tot_dfs[(tot_dfs.name==individual) & (tot_dfs.batch==batch)].sort_values(by="aug").set_index("aug")
        scaler = MinMaxScaler()
        subset_df["snip"] = scaler.fit_transform(subset_df[["snip"]])
        rank_batches_snip.append(subset_df["snip"])
rank_batches_snip=pd.concat(rank_batches_snip)
rank_batches_snip=rank_batches_snip.reset_index().groupby("aug").mean().sort_values(by="snip", ascending=True)

rank_batches_grad=[]
for individual in individuals:
    for batch in range(num_batches):
        #rank_batches_grad.append(tot_dfs[(tot_dfs.name==individual) & (tot_dfs.batch==batch)].sort_values(by="aug").set_index("aug")["grad_norm"].rank())
        subset_df = tot_dfs[(tot_dfs.name==individual) & (tot_dfs.batch==batch)].sort_values(by="aug").set_index("aug")
        scaler = MinMaxScaler()
        subset_df["grad_norm"] = scaler.fit_transform(subset_df[["grad_norm"]])
        rank_batches_grad.append(subset_df["grad_norm"])
rank_batches_grad=pd.concat(rank_batches_grad)
rank_batches_grad=rank_batches_grad.reset_index().groupby("aug").mean().sort_values(by="grad_norm", ascending=True)
#"grad_norm", "snip", "synflow"

KeyError: "None of [Index(['synflow'], dtype='object')] are in the [columns]"

In [ ]:
#pd.read_csv("/home/woody/iwb3/iwb3021h/THESIS_RESULTS/5batches_fisher/fisher_LaMelo.csv", index_col=0)

In [28]:
#rank_batches=pd.concat([rank_batches_fisher,rank_batches_jac,rank_batches_plain,rank_batches_syn, rank_batches_snip,rank_batches_grad],axis=1)
rank_batches=pd.concat([rank_batches_fisher,rank_batches_jac], axis=1)
#rank_batches=pd.concat([rank_batches_fisher], axis=1)
rank_batches["total_rank"]=rank_batches["fisher"]+rank_batches["jacob_cov"]#-rank_batches["plain"]#-rank_batches["synflow"]
#rank_batches["total_rank"]=rank_batches["fisher"]+rank_batches["jacob_cov"]-rank_batches["snip"]#-rank_batches["grad_norm"]+rank_batches["plain"]
#rank_batches["total_rank"]=rank_batches["jacob_cov"]#-rank_batches["snip"]-rank_batches["grad_norm"]




In [29]:
#pd.merge(rank_batches.reset_index(), test_scores_df, on="aug").sort_values(by="test_acc",ascending=False).to_csv("zcost_proxies/results/augs/ranks_mateo.csv")

In [30]:
test_scores_df.sort_values(by="test_acc") #gutenberg

,aug,test_acc,val_acc,train_acc,epoch_time
20,20,44.65,65.949997,92.868591,22.571819
18,18,61.06,70.889999,86.999199,20.804323
16,16,65.09,75.970001,98.437500,18.008868
21,21,65.50,74.639999,83.475563,22.699272
15,15,70.36,71.459999,96.937103,17.953436
17,17,75.98,81.369995,97.858574,18.079419
19,19,80.66,85.229996,93.471558,20.538386
8,8,91.24,96.110001,99.671478,11.757495
3,3,91.72,96.070000,95.140228,32.333925
7,7,91.74,96.649994,99.505211,73.663493


In [17]:
rank_batches.sort_values(by="total_rank") #Mateo

,fisher,jacob_cov,total_rank
aug,,,
5,0.253234,0.154715,0.407949
2,0.395585,0.242042,0.637627
4,0.267752,0.431610,0.699362
1,0.393258,0.332143,0.725401
3,0.671429,0.274998,0.946427
6,0.365678,0.873929,1.239608
14,0.502352,0.811039,1.313391
11,0.510959,0.813141,1.324100
12,0.526594,0.802584,1.329179


In [29]:
test_scores_df.sort_values(by="test_acc") #mateo

,aug,test_acc,val_acc,train_acc,epoch_time
20,20,44.65,65.949997,92.868591,22.571819
18,18,61.06,70.889999,86.999199,20.804323
16,16,65.09,75.970001,98.437500,18.008868
21,21,65.50,74.639999,83.475563,22.699272
15,15,70.36,71.459999,96.937103,17.953436
17,17,75.98,81.369995,97.858574,18.079419
19,19,80.66,85.229996,93.471558,20.538386
8,8,91.24,96.110001,99.671478,11.757495
3,3,91.72,96.070000,95.140228,32.333925
7,7,91.74,96.649994,99.505211,73.663493


In [31]:
rank_batches.sort_values(by="total_rank").head() #Mateo

,fisher,jacob_cov,total_rank
aug,,,
4,0.138536,0.273551,0.412087
5,0.130735,0.461898,0.592632
3,0.081480,0.595937,0.677417
1,0.084324,0.614075,0.698399
2,0.099675,0.614542,0.714217
7,0.274085,0.599138,0.873223
6,0.284382,0.636705,0.921087
0,0.688985,0.252697,0.941681
10,0.678970,0.298314,0.977284


In [64]:
test_scores_df.sort_values(by="test_acc") #sadie

,aug,test_acc,val_acc,train_acc,epoch_time
17,17,52.542567,51.986340,76.720123,28.325682
19,19,57.387727,57.324986,85.108276,29.629261
16,16,57.741972,55.936256,97.644829,27.771861
15,15,58.873272,58.542973,92.116684,27.501107
21,21,65.238258,62.287991,84.333885,31.950643
18,18,65.763913,67.706314,95.225693,30.304365
20,20,66.918067,62.424587,82.118057,31.795406
7,7,82.036339,77.643707,85.448647,64.358486
6,6,82.322020,78.281158,81.140350,68.653077
0,0,83.533310,76.550941,88.041389,8.871612


In [81]:
rank_batches.sort_values(by="total_rank") #sadie

,fisher,jacob_cov,plain,total_rank
aug,,,,
9,0.394210,0.229126,0.405355,0.623336
5,0.222800,0.400817,0.415223,0.623617
8,0.385125,0.287044,0.396768,0.672169
0,0.379953,0.301777,0.450394,0.681730
10,0.371916,0.310991,0.458079,0.682907
2,0.253766,0.434494,0.458612,0.688260
4,0.403361,0.354616,0.434397,0.757977
3,0.377520,0.420317,0.468223,0.797837
1,0.387831,0.462285,0.441908,0.850116


In [25]:
test_scores_df.sort_values(by="test_acc") #caitie

,aug,test_acc,val_acc,train_acc,epoch_time
21,21,24.76,26.413334,36.549591,34.500305
17,17,26.16,29.153334,46.754807,30.988798
19,19,26.22,26.980000,41.893696,32.389005
15,15,26.39,28.760000,38.991276,29.990626
16,16,26.54,30.646667,38.575054,30.358766
20,20,26.61,26.133333,42.930912,34.398647
18,18,27.78,32.346668,50.367252,32.821100
7,7,44.88,47.466667,50.520832,66.751601
0,0,45.27,46.833332,55.259525,9.502931
6,6,45.32,47.613335,61.037659,69.354500


In [50]:
rank_batches.sort_values(by="total_rank") #caitie

,fisher,jacob_cov,plain,total_rank
aug,,,,
5,0.129643,0.250795,0.460523,0.380438
1,0.096113,0.373394,0.429324,0.469507
3,0.127358,0.378877,0.492584,0.506234
4,0.116183,0.402736,0.406470,0.518919
2,0.112871,0.514276,0.470087,0.627147
6,0.165445,0.488894,0.433816,0.654338
7,0.177791,0.520310,0.410865,0.698101
9,0.261950,0.499761,0.499549,0.761710
8,0.271501,0.490523,0.475759,0.762024


In [24]:
test_scores_df.sort_values(by="test_acc") #sokoto

,aug,test_acc,val_acc,train_acc,epoch_time
7,7,17.24,18.230000,31.161860,63.783878
3,3,19.09,20.820000,31.522436,33.063324
2,2,19.36,20.400000,28.267229,33.193070
6,6,19.40,19.459999,28.455530,64.536691
0,0,19.44,19.830000,37.433895,18.563603
16,16,20.66,20.769999,29.985979,24.592947
17,17,21.22,21.219999,25.220354,24.556963
15,15,22.89,22.809999,38.343349,24.585550
19,19,26.93,26.299999,33.072918,26.992012
4,4,27.02,27.480000,25.030048,27.907956


In [48]:
rank_batches.sort_values(by="total_rank") #sokoto

,fisher,jacob_cov,plain,total_rank
aug,,,,
1,0.182838,0.418948,0.424737,-0.241899
5,0.345351,0.596175,0.460205,-0.114854
4,0.350270,0.575440,0.429830,-0.079559
2,0.433884,0.426471,0.502023,-0.068139
9,0.552933,0.367249,0.526285,0.026648
19,0.506566,0.449081,0.469006,0.037560
21,0.525771,0.421524,0.461621,0.064150
16,0.595754,0.453121,0.531290,0.064464
10,0.555893,0.361669,0.468389,0.087504


In [43]:
rank_batches.sort_values(by="total_rank") #sokoto

,fisher,jacob_cov,total_rank
aug,,,
1,0.182838,0.418948,0.601786
2,0.433884,0.426471,0.860356
10,0.555893,0.361669,0.917562
9,0.552933,0.367249,0.920182
4,0.350270,0.575440,0.925710
5,0.345351,0.596175,0.941526
21,0.525771,0.421524,0.947295
19,0.506566,0.449081,0.955647
6,0.572223,0.408569,0.980792


In [107]:
test_scores_df.sort_values(by="test_acc") #adaline

,aug,test_acc,val_acc,train_acc,epoch_time
21,21,5.12,10.840000,55.956196,26.748872
18,18,10.77,15.593333,86.554039,24.910084
15,15,19.43,26.533333,95.014244,22.396068
20,20,27.07,30.953333,93.035522,26.579179
16,16,29.57,33.786667,97.936699,22.453781
19,19,30.84,40.893333,89.320694,24.480816
17,17,41.25,44.540001,97.856567,22.630426
3,3,91.19,90.800003,96.552261,28.616787
0,0,91.24,90.900002,99.964386,14.214638
7,7,91.74,91.146667,99.568199,61.351543


In [115]:
rank_batches.sort_values(by="total_rank") #Adaline

,fisher,jacob_cov,plain,total_rank
aug,,,,
3,0.061751,0.641191,0.510714,0.061751
2,0.068627,0.607711,0.504580,0.068627
1,0.079748,0.660396,0.521054,0.079748
4,0.125358,0.399214,0.518572,0.125358
5,0.133674,0.397011,0.483022,0.133674
7,0.281509,0.551586,0.494619,0.281509
6,0.281871,0.632311,0.515811,0.281871
14,0.651571,0.623513,0.443225,0.651571
9,0.664694,0.466241,0.406447,0.664694


In [111]:
rank_batches.sort_values(by="total_rank") #Adaline

,fisher,jacob_cov,plain,total_rank
aug,,,,
4,0.125358,0.399214,0.518572,0.006000
5,0.133674,0.397011,0.483022,0.047663
2,0.068627,0.607711,0.504580,0.171758
3,0.061751,0.641191,0.510714,0.192228
1,0.079748,0.660396,0.521054,0.219091
7,0.281509,0.551586,0.494619,0.338475
6,0.281871,0.632311,0.515811,0.398370
0,0.679702,0.385504,0.505897,0.559308
10,0.698596,0.315535,0.400370,0.613761


In [137]:
test_scores_df.sort_values(by="test_acc") #lamelo

,aug,test_acc,val_acc,train_acc,epoch_time
13,13,75.130,74.724998,81.701725,23.656474
14,14,77.335,76.805000,82.713345,24.366148
12,12,79.090,78.084995,87.545078,20.956593
11,11,79.110,79.124996,93.925285,23.134880
20,20,79.665,80.189999,84.072517,27.438826
8,8,81.220,81.084995,99.848763,19.322388
21,21,82.080,82.024998,93.892231,27.866150
9,9,82.230,80.264999,93.528652,21.274822
18,18,82.550,82.250000,94.988983,25.855804
1,1,84.045,81.264996,71.393234,30.812956


In [150]:
rank_batches.sort_values(by="total_rank") #lamelo

,fisher,jacob_cov,plain,total_rank
aug,,,,
5,0.213520,0.234595,0.405188,0.448115
1,0.326843,0.195249,0.427526,0.522092
4,0.351119,0.332180,0.505077,0.683298
2,0.431556,0.308832,0.504442,0.740388
3,0.606822,0.212668,0.482542,0.819490
6,0.201837,0.899847,0.430676,1.101684
20,0.583006,0.835670,0.463188,1.418676
12,0.636893,0.809334,0.453831,1.446227
7,0.597248,0.849536,0.519647,1.446784


In [169]:
test_scores_df.sort_values(by="test_acc") #chester

,aug,test_acc,val_acc,train_acc,epoch_time
10,17,48.974897,50.800079,48.439505,23.856391
9,16,51.610161,53.775377,56.492390,23.826331
8,15,51.985199,53.660366,58.344353,23.835406
3,10,53.175318,52.120213,82.289665,19.437332
0,0,53.485349,51.885189,93.808098,17.108011
12,19,54.095410,54.300430,58.433496,25.992771
11,18,56.855686,57.375738,54.357973,26.540709
13,20,57.385739,57.345736,53.901245,28.244505
14,21,57.455746,56.020603,55.656052,28.411490
4,11,58.660866,57.685770,62.024240,23.829775


In [186]:
rank_batches.sort_values(by="total_rank") #chester

,fisher,jacob_cov,total_rank
aug,,,
11,0.329809,0.249396,0.579205
13,0.363135,0.353767,0.716903
14,0.480931,0.284233,0.765164
12,0.371865,0.413237,0.785102
20,0.490468,0.376449,0.866917
10,0.490154,0.402051,0.892204
9,0.489038,0.431998,0.921036
0,0.478331,0.458759,0.937090
8,0.443916,0.507070,0.950986


In [18]:
from scipy.stats import spearmanr # Gutenberg
spearmanr(test_scores_df.sort_values(by="aug",ascending=False)["test_acc"].values, rank_batches.reset_index().sort_values(by="aug",ascending=False)["total_rank"].values)

SignificanceResult(statistic=-0.30217453901947683, pvalue=0.17168912441587655)

In [44]:
from scipy.stats import spearmanr # Mateo
spearmanr(test_scores_df.sort_values(by="aug",ascending=False)["test_acc"].values, rank_batches.reset_index().sort_values(by="aug",ascending=False)["total_rank"].values)

SignificanceResult(statistic=-0.6194240542066629, pvalue=0.002110576672057838)

In [82]:
from scipy.stats import spearmanr # sadie
spearmanr(test_scores_df.sort_values(by="aug",ascending=False)["test_acc"].values, rank_batches.reset_index().sort_values(by="aug",ascending=False)["total_rank"].values)

SignificanceResult(statistic=-0.7402597402597403, pvalue=8.185823879055852e-05)

In [45]:
from scipy.stats import spearmanr # Caitie
spearmanr(test_scores_df.sort_values(by="aug",ascending=False)["test_acc"].values, rank_batches.reset_index().sort_values(by="aug",ascending=False)["total_rank"].values)

SignificanceResult(statistic=-0.5143986448334276, pvalue=0.014312414476149815)

In [187]:
from scipy.stats import spearmanr # Chester
spearmanr(test_scores_df.sort_values(by="aug",ascending=False)["test_acc"].values, rank_batches.reset_index().sort_values(by="aug",ascending=False)["total_rank"].values)

SignificanceResult(statistic=-0.7392857142857142, pvalue=0.0016355111435407642)

In [155]:
from scipy.stats import spearmanr # LaMelo
spearmanr(test_scores_df.sort_values(by="aug",ascending=False)["test_acc"].values, rank_batches.reset_index().sort_values(by="aug",ascending=False)["total_rank"].values)

SignificanceResult(statistic=0.044607566346696784, pvalue=0.8437394727309631)

In [113]:
from scipy.stats import spearmanr # Adaline
spearmanr(test_scores_df.sort_values(by="aug",ascending=False)["test_acc"].values, rank_batches.reset_index().sort_values(by="aug",ascending=False)["total_rank"].values)

SignificanceResult(statistic=-0.39920948616600793, pvalue=0.065688713148864)

In [37]:
from scipy.stats import spearmanr # Csokoto
spearmanr(test_scores_df.sort_values(by="aug",ascending=False)["test_acc"].values, rank_batches.reset_index().sort_values(by="aug",ascending=False)["total_rank"].values)

SignificanceResult(statistic=0.15979672501411638, pvalue=0.4774887142899461)

In [24]:
rank_batches.reset_index().groupby("aug").sum().sort_values(by="jacob_cov")

,jacob_cov
aug,
10,192.0
0,211.0
4,248.0
9,299.0
8,300.0
5,327.0
7,410.0
19,458.0
2,460.0


In [21]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
tot_dfs=pd.read_csv("zcost_proxies/results/augs/mateo_zcosts.csv", index_col=0)



In [8]:
tot_dfs_iter=tot_dfs.groupby(["name", "aug"]).agg(['mean', 'std']).reset_index()

In [15]:
#tot_dfs.to_csv("gutenberg_zcosts.csv")

In [14]:
tot_dfs

,name,fisher,jacob_cov,plain,aug,batch
0,rich_taipan,0.314766+ 0.000000j,-4.799617e+06-4.235165e- 22j,0.279924+0.000000j,0,0
1,dazzling_anaconda,0.084117+ 0.000000j,-4.799625e+06-4.235165e- 22j,0.102339+0.000000j,0,0
2,purring_meerkat,0.091622+ 0.000000j,-4.799619e+06-0.000000e+ 00j,0.232129+0.000000j,0,0
3,ultraviolet_gopher,52.119835+ 0.000000j,-4.799623e+06-0.000000e+ 00j,1.332811+0.000000j,0,0
4,dancing_nautilus,2.662895+ 0.000000j,-4.799659e+06-8.470329e- 22j,-0.299878+0.000000j,0,0
...,...,...,...,...,...,...
5,classy_cricket,0.166066+ 0.000000j,-4.799639e+06-1.694066e- 21j,0.212949+0.000000j,21,4
6,prophetic_bird,124.075615+ 0.000000j,-4.799609e+06-1.270549e- 21j,-0.813117+0.000000j,21,4
7,whimsical_beetle,0.276991+ 0.000000j,-4.799621e+06-8.470329e- 22j,0.190989+0.000000j,21,4
8,adamant_mouse,0.026611+ 0.000000j,-4.799623e+06-1.694066e- 21j,0.072604+0.000000j,21,4


In [170]:
#tot_dfs_iter.to_csv("adaline_zcosts.csv")
#tot_dfs_iter=pd.read_csv("adaline_zcosts.csv", index_col=0).iloc[1:,:]
#tot_dfs_iter.columns=["name","aug","fishermean","fisherstd","jacob_covmean","jacob_covstd","plainmean","plainstd","batchmean","batchstd"]
#tot_dfs_iter.aug=tot_dfs_iter.aug.astype(int)
#tot_dfs_iter["jacob_covmean"]=tot_dfs_iter["jacob_covmean"].astype(float)
#tot_dfs_iter["fishermean"]=tot_dfs_iter["fishermean"].astype(float)
#tot_dfs_iter["jacob_covstd"]=tot_dfs_iter["jacob_covstd"].astype(float)

In [9]:
tot_dfs_iter.columns = [''.join(col).strip() for col in tot_dfs_iter.columns.values]

In [10]:
merged_df=pd.merge(test_scores_df, tot_dfs_iter,on="aug" )

In [11]:
merged_df

,aug,test_acc,val_acc,train_acc,epoch_time,name,fishermean,fisherstd,jacob_covmean,jacob_covstd,plainmean,plainstd,batchmean,batchstd
0,0,42.7,40.600002,62.005875,17.563438,amphibian_copperhead,0.323881,0.050548,-173.586132,5.002655,0.141747,0.053053,2.0,1.581139
1,0,42.7,40.600002,62.005875,17.563438,delectable_octopus,0.513576,0.029283,-171.775537,5.904553,0.084783,0.162273,2.0,1.581139
2,0,42.7,40.600002,62.005875,17.563438,golden_scorpion,0.215012,0.016343,-175.370866,4.364170,0.179489,0.149683,2.0,1.581139
3,0,42.7,40.600002,62.005875,17.563438,invaluable_leopard,0.034849,0.001394,-170.227144,2.875859,0.159882,0.107546,2.0,1.581139
4,0,42.7,40.600002,62.005875,17.563438,juicy_uakari,0.069439,0.007556,-169.500420,1.887142,0.248043,0.054760,2.0,1.581139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,21,44.4,43.626667,55.847134,27.494028,mega_frog,0.281868,0.026602,-167.298168,1.939449,0.348036,0.368770,2.0,1.581139
216,21,44.4,43.626667,55.847134,27.494028,prudent_crab,0.356618,0.024802,-174.582007,3.723874,0.255756,0.175251,2.0,1.581139
217,21,44.4,43.626667,55.847134,27.494028,splendid_duck,0.154497,0.013924,-169.666208,3.207041,-0.013344,0.044417,2.0,1.581139
218,21,44.4,43.626667,55.847134,27.494028,tidy_silkworm,1.000554,0.123674,-168.963242,7.013945,0.009873,0.161105,2.0,1.581139


In [79]:
tot_dfs=tot_dfs_iter[["name","aug","fishermean"]].rename(columns={"fishermean":"fisher"})

In [12]:
corr_matrix = merged_df.drop(columns=["name"]).corr(method='spearman')

In [13]:
corr_matrix #Gutenberg

,aug,test_acc,val_acc,train_acc,epoch_time,fishermean,fisherstd,jacob_covmean,jacob_covstd,plainmean,plainstd,batchmean,batchstd
aug,1.000000,-0.032194,0.546019,0.132693,-0.153021,0.081791,-0.206068,0.608637,-0.430773,0.054772,0.106294,NaN,NaN
test_acc,-0.032194,1.000000,0.542785,-0.451285,0.274499,-0.073119,0.171228,-0.291461,0.257020,0.011460,0.005812,NaN,NaN
val_acc,0.546019,0.542785,1.000000,-0.321287,0.167702,-0.024616,0.019066,0.221681,-0.133517,0.018141,0.078282,NaN,NaN
train_acc,0.132693,-0.451285,-0.321287,1.000000,-0.366460,0.105877,-0.213965,0.232963,-0.178236,0.025609,-0.011236,NaN,NaN
epoch_time,-0.153021,0.274499,0.167702,-0.366460,1.000000,-0.105493,0.148882,-0.244594,0.189146,0.035480,0.069505,NaN,NaN
fishermean,0.081791,-0.073119,-0.024616,0.105877,-0.105493,1.000000,0.773791,0.165427,-0.020133,0.182138,0.644710,NaN,NaN
fisherstd,-0.206068,0.171228,0.019066,-0.213965,0.148882,0.773791,1.000000,-0.158403,0.214764,0.160076,0.611942,NaN,NaN
jacob_covmean,0.608637,-0.291461,0.221681,0.232963,-0.244594,0.165427,-0.158403,1.000000,-0.763598,0.118107,0.149357,NaN,NaN
jacob_covstd,-0.430773,0.257020,-0.133517,-0.178236,0.189146,-0.020133,0.214764,-0.763598,1.000000,-0.098295,-0.064460,NaN,NaN
plainmean,0.054772,0.011460,0.018141,0.025609,0.035480,0.182138,0.160076,0.118107,-0.098295,1.000000,0.287252,NaN,NaN


In [26]:
corr_matrix #Mateo

,aug,test_acc,val_acc,train_acc,epoch_time,fishermean,fisherstd,jacob_covmean,jacob_covstd,plainmean,plainstd,batchmean,batchstd
aug,1.000000,-0.612648,-0.602484,-0.197064,-0.192547,0.122923,0.060379,0.443013,-0.251058,-0.046491,0.031836,NaN,NaN
test_acc,-0.612648,1.000000,0.949181,0.129305,-0.032185,-0.095566,-0.049481,-0.299524,0.170339,0.003283,-0.000970,NaN,NaN
val_acc,-0.602484,0.949181,1.000000,0.169960,0.102202,-0.108754,-0.058055,-0.268680,0.165206,0.021142,-0.004682,NaN,NaN
train_acc,-0.197064,0.129305,0.169960,1.000000,-0.443252,0.051884,0.055200,-0.020702,-0.036778,-0.014328,0.002279,NaN,NaN
epoch_time,-0.192547,-0.032185,0.102202,-0.443252,1.000000,-0.122901,-0.081103,0.234170,-0.032840,0.030449,-0.018502,NaN,NaN
fishermean,0.122923,-0.095566,-0.108754,0.051884,-0.122901,1.000000,0.973801,0.035840,0.099897,0.386477,0.937151,NaN,NaN
fisherstd,0.060379,-0.049481,-0.058055,0.055200,-0.081103,0.973801,1.000000,-0.005022,0.130089,0.365787,0.939283,NaN,NaN
jacob_covmean,0.443013,-0.299524,-0.268680,-0.020702,0.234170,0.035840,-0.005022,1.000000,-0.495759,0.093081,0.001678,NaN,NaN
jacob_covstd,-0.251058,0.170339,0.165206,-0.036778,-0.032840,0.099897,0.130089,-0.495759,1.000000,0.001345,0.122217,NaN,NaN
plainmean,-0.046491,0.003283,0.021142,-0.014328,0.030449,0.386477,0.365787,0.093081,0.001345,1.000000,0.400199,NaN,NaN


In [61]:
corr_matrix #sadie

,aug,test_acc,val_acc,train_acc,epoch_time,fishermean,fisherstd,jacob_covmean,jacob_covstd,plainmean,plainstd,batchmean,batchstd
aug,1.000000,-0.584416,-0.546019,0.088650,0.311124,0.070949,-0.028847,0.335625,-0.146513,0.088142,0.070994,NaN,NaN
test_acc,-0.584416,1.000000,0.949181,0.207228,-0.571993,-0.067283,0.048533,-0.277119,0.289077,-0.119776,-0.071976,NaN,NaN
val_acc,-0.546019,0.949181,1.000000,0.360813,-0.558442,-0.054591,0.055110,-0.256677,0.303811,-0.132704,-0.064169,NaN,NaN
train_acc,0.088650,0.207228,0.360813,1.000000,-0.507623,0.019449,0.026229,-0.034668,0.120148,-0.030370,0.055178,NaN,NaN
epoch_time,0.311124,-0.571993,-0.558442,-0.507623,1.000000,0.047033,-0.034273,0.186156,-0.265172,0.054591,0.045194,NaN,NaN
fishermean,0.070949,-0.067283,-0.054591,0.019449,0.047033,1.000000,0.957521,-0.692159,0.054864,0.418689,0.840425,NaN,NaN
fisherstd,-0.028847,0.048533,0.055110,0.026229,-0.034273,0.957521,1.000000,-0.712166,0.089531,0.367142,0.813510,NaN,NaN
jacob_covmean,0.335625,-0.277119,-0.256677,-0.034668,0.186156,-0.692159,-0.712166,1.000000,-0.346354,-0.218100,-0.581744,NaN,NaN
jacob_covstd,-0.146513,0.289077,0.303811,0.120148,-0.265172,0.054864,0.089531,-0.346354,1.000000,-0.066209,0.009415,NaN,NaN
plainmean,0.088142,-0.119776,-0.132704,-0.030370,0.054591,0.418689,0.367142,-0.218100,-0.066209,1.000000,0.410807,NaN,NaN


In [22]:
corr_matrix #caitie

,aug,test_acc,val_acc,train_acc,epoch_time,fishermean,fisherstd,jacob_covmean,jacob_covstd,plainmean,plainstd,batchmean,batchstd
aug,1.000000,-0.452287,-0.455675,-0.456804,0.348391,0.160637,0.136675,0.175382,-0.194922,-0.030471,0.016505,NaN,NaN
test_acc,-0.452287,1.000000,0.975155,0.944664,-0.622812,-0.130493,-0.118185,-0.071942,0.124503,-0.016787,-0.019427,NaN,NaN
val_acc,-0.455675,0.975155,1.000000,0.926595,-0.614907,-0.128180,-0.120058,-0.057220,0.141402,-0.028779,-0.020013,NaN,NaN
train_acc,-0.456804,0.944664,0.926595,1.000000,-0.608131,-0.123284,-0.108178,-0.035345,0.065151,-0.027775,-0.026782,NaN,NaN
epoch_time,0.348391,-0.622812,-0.614907,-0.608131,1.000000,0.055505,0.025316,0.012432,-0.131790,-0.038662,-0.019946,NaN,NaN
fishermean,0.160637,-0.130493,-0.128180,-0.123284,0.055505,1.000000,0.977088,-0.110465,-0.188222,0.415389,0.933160,NaN,NaN
fisherstd,0.136675,-0.118185,-0.120058,-0.108178,0.025316,0.977088,1.000000,-0.164839,-0.149627,0.397836,0.913996,NaN,NaN
jacob_covmean,0.175382,-0.071942,-0.057220,-0.035345,0.012432,-0.110465,-0.164839,1.000000,-0.575444,0.228511,-0.071381,NaN,NaN
jacob_covstd,-0.194922,0.124503,0.141402,0.065151,-0.131790,-0.188222,-0.149627,-0.575444,1.000000,-0.225411,-0.193311,NaN,NaN
plainmean,-0.030471,-0.016787,-0.028779,-0.027775,-0.038662,0.415389,0.397836,0.228511,-0.225411,1.000000,0.459266,NaN,NaN


In [26]:
corr_matrix #sokot

,aug,test_acc,val_acc,train_acc,epoch_time,fishermean,fisherstd,jacob_covmean,jacob_covstd,plainmean,plainstd,grad_normmean,grad_normstd,snipmean,snipstd,synflowmean,synflowstd,batchmean,batchstd
aug,1.000000,0.481649,0.354037,0.263693,-0.171090,0.050397,-0.021629,0.049568,-0.000388,-0.049463,-0.011176,0.047136,-0.045955,0.046202,-0.038886,0.000141,0.017610,NaN,NaN
test_acc,0.481649,1.000000,0.967250,0.570864,-0.293055,0.028997,-0.002309,0.067566,-0.032999,-0.030178,0.000582,0.024643,0.051208,0.027111,0.011846,-0.032399,0.028609,NaN,NaN
val_acc,0.354037,0.967250,1.000000,0.622812,-0.255788,0.029737,0.000599,0.080134,-0.039380,-0.021171,0.001128,0.024326,0.052477,0.027499,0.011264,-0.032152,0.026124,NaN,NaN
train_acc,0.263693,0.570864,0.622812,1.000000,-0.357425,0.070122,-0.031835,0.122705,-0.048352,0.061678,-0.029614,0.067795,-0.053006,0.061855,-0.061678,-0.004795,-0.000370,NaN,NaN
epoch_time,-0.171090,-0.293055,-0.255788,-0.357425,1.000000,-0.055773,0.027305,-0.113856,0.026247,-0.048669,0.014031,-0.050926,0.011088,-0.052970,0.009025,0.010806,0.000670,NaN,NaN
fishermean,0.050397,0.028997,0.029737,0.070122,-0.055773,1.000000,0.908393,-0.376835,0.173449,0.072032,0.850163,0.986181,0.858644,0.994440,0.872300,0.602375,0.591211,NaN,NaN
fisherstd,-0.021629,-0.002309,0.000599,-0.031835,0.027305,0.908393,1.000000,-0.403544,0.164700,0.048983,0.848706,0.912578,0.944008,0.909833,0.961222,0.622945,0.619472,NaN,NaN
jacob_covmean,0.049568,0.067566,0.080134,0.122705,-0.113856,-0.376835,-0.403544,1.000000,-0.435538,0.114589,-0.389938,-0.339703,-0.374825,-0.380000,-0.345358,0.169786,0.182529,NaN,NaN
jacob_covstd,-0.000388,-0.032999,-0.039380,-0.048352,0.026247,0.173449,0.164700,-0.435538,1.000000,-0.025904,0.161640,0.172311,0.181446,0.170797,0.153628,-0.024812,-0.038499,NaN,NaN
plainmean,-0.049463,-0.030178,-0.021171,0.061678,-0.048669,0.072032,0.048983,0.114589,-0.025904,1.000000,0.009252,0.075952,0.061580,0.055556,0.074821,0.172311,0.190022,NaN,NaN


In [166]:
corr_matrix # Chester

,aug,test_acc,val_acc,train_acc,epoch_time,fishermean,fisherstd,jacob_covmean,jacob_covstd,plainmean,plainstd,grad_normmean,grad_normstd,snipmean,snipstd,synflowmean,synflowstd,batchmean,batchstd
aug,1.000000,-0.275000,-0.092857,-0.878571,0.896429,0.046438,-0.027284,0.339044,-0.142544,-0.041705,-0.001114,-0.003007,-0.050280,0.017094,0.005123,0.000724,0.019600,NaN,NaN
test_acc,-0.275000,1.000000,0.907143,0.371429,0.089286,-0.071885,0.026115,-0.474962,0.122221,-0.156465,-0.020157,0.026727,0.027562,-0.015925,-0.017094,0.015090,-0.022328,NaN,NaN
val_acc,-0.092857,0.907143,1.000000,0.160714,0.214286,-0.073555,0.003619,-0.391106,0.110138,-0.236980,-0.036471,0.014923,-0.002951,-0.022273,-0.006960,0.007740,-0.011470,NaN,NaN
train_acc,-0.878571,0.371429,0.160714,1.000000,-0.792857,-0.047051,0.036360,-0.384925,0.117321,-0.005457,0.036304,0.015646,0.021549,-0.018152,-0.024333,0.008408,-0.026115,NaN,NaN
epoch_time,0.896429,0.089286,0.214286,-0.792857,1.000000,0.022774,-0.011749,0.154683,-0.126731,-0.042819,-0.007851,0.008074,-0.023442,0.007406,-0.009076,0.008185,0.004733,NaN,NaN
fishermean,0.046438,-0.071885,-0.073555,-0.047051,0.022774,1.000000,0.933218,0.494152,-0.199729,-0.004716,0.897882,0.978110,0.916564,0.975109,0.918967,0.729273,0.731516,NaN,NaN
fisherstd,-0.027284,0.026115,0.003619,0.036360,-0.011749,0.933218,1.000000,0.381346,-0.176978,-0.060476,0.894590,0.916071,0.937954,0.912688,0.924398,0.812390,0.815522,NaN,NaN
jacob_covmean,0.339044,-0.474962,-0.391106,-0.384925,0.154683,0.494152,0.381346,1.000000,-0.183575,-0.025731,0.331065,0.425273,0.362185,0.450288,0.402090,0.269033,0.308396,NaN,NaN
jacob_covstd,-0.142544,0.122221,0.110138,0.117321,-0.126731,-0.199729,-0.176978,-0.183575,1.000000,0.091159,-0.173616,-0.152795,-0.163768,-0.186360,-0.188305,-0.171498,-0.159907,NaN,NaN
plainmean,-0.041705,-0.156465,-0.236980,-0.005457,-0.042819,-0.004716,-0.060476,-0.025731,0.091159,1.000000,-0.025606,-0.006438,-0.003695,-0.010827,-0.062504,-0.038433,-0.036739,NaN,NaN


In [134]:
corr_matrix # LaMelo

,aug,test_acc,val_acc,train_acc,epoch_time,fishermean,fisherstd,jacob_covmean,jacob_covstd,plainmean,plainstd,batchmean,batchstd
aug,1.000000,-0.584416,-0.546019,0.088650,0.311124,0.070949,-0.028847,0.335625,-0.146513,0.088142,0.070994,NaN,NaN
test_acc,-0.584416,1.000000,0.949181,0.207228,-0.571993,-0.067283,0.048533,-0.277119,0.289077,-0.119776,-0.071976,NaN,NaN
val_acc,-0.546019,0.949181,1.000000,0.360813,-0.558442,-0.054591,0.055110,-0.256677,0.303811,-0.132704,-0.064169,NaN,NaN
train_acc,0.088650,0.207228,0.360813,1.000000,-0.507623,0.019449,0.026229,-0.034668,0.120148,-0.030370,0.055178,NaN,NaN
epoch_time,0.311124,-0.571993,-0.558442,-0.507623,1.000000,0.047033,-0.034273,0.186156,-0.265172,0.054591,0.045194,NaN,NaN
fishermean,0.070949,-0.067283,-0.054591,0.019449,0.047033,1.000000,0.957521,-0.692159,0.054864,0.418689,0.840425,NaN,NaN
fisherstd,-0.028847,0.048533,0.055110,0.026229,-0.034273,0.957521,1.000000,-0.712166,0.089531,0.367142,0.813510,NaN,NaN
jacob_covmean,0.335625,-0.277119,-0.256677,-0.034668,0.186156,-0.692159,-0.712166,1.000000,-0.346354,-0.218100,-0.581744,NaN,NaN
jacob_covstd,-0.146513,0.289077,0.303811,0.120148,-0.265172,0.054864,0.089531,-0.346354,1.000000,-0.066209,0.009415,NaN,NaN
plainmean,0.088142,-0.119776,-0.132704,-0.030370,0.054591,0.418689,0.367142,-0.218100,-0.066209,1.000000,0.410807,NaN,NaN


In [76]:
corr_matrix # Adaline

,aug,test_acc,fishermean,fisherstd,jacob_covmean,jacob_covstd,plainmean,plainstd,batchmean,batchstd
aug,1.000000,-0.585545,0.170122,0.124626,0.337125,-0.154187,-0.112463,0.034444,NaN,NaN
test_acc,-0.585545,1.000000,-0.099295,-0.049604,-0.197798,0.110242,0.061132,-0.043275,NaN,NaN
fishermean,0.170122,-0.099295,1.000000,0.934704,0.009135,-0.086888,0.326628,0.885085,NaN,NaN
fisherstd,0.124626,-0.049604,0.934704,1.000000,-0.144035,0.013779,0.308052,0.825188,NaN,NaN
jacob_covmean,0.337125,-0.197798,0.009135,-0.144035,1.000000,-0.585606,0.009296,-0.069507,NaN,NaN
jacob_covstd,-0.154187,0.110242,-0.086888,0.013779,-0.585606,1.000000,-0.058516,-0.044180,NaN,NaN
plainmean,-0.112463,0.061132,0.326628,0.308052,0.009296,-0.058516,1.000000,0.392027,NaN,NaN
plainstd,0.034444,-0.043275,0.885085,0.825188,-0.069507,-0.044180,0.392027,1.000000,NaN,NaN
batchmean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
batchstd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
dataset="Mateo"

In [23]:
dataset

'Mateo'

In [24]:
import pandas as pd
test_scores_df=[]
for i in range(23):
    try:
        #df_aux=pd.read_csv(f"/home/woody/iwb3/iwb3021h/THESIS_RESULTS/full_training_evonas/finetuning/{dataset}/augmentations_test_1/{dataset}/aug_{i}/test_acc.csv", index_col=0)
        #df_aux["aug"]=i
        #test_scores_df.append(df_aux)
        
        #df_aux["model"]="regnet"
        
        #df_aux2=pd.read_csv(f"/home/woody/iwb3/iwb3021h/THESIS_RESULTS/full_training_evonas/finetuning/{dataset}/augmentations_test_2/{dataset}/aug_{i}/test_acc.csv", index_col=0)
        #test_scores_df.append(df_aux2)
        #df_aux2["aug"]=i
        
        df_aux3=pd.read_csv(f"/home/woody/iwb3/iwb3021h/THESIS_RESULTS/full_training_evonas/finetuning/{dataset}/augmentations_test_resnet/{dataset}/aug_{i}/test_acc.csv", index_col=0)
        test_scores_df.append(df_aux3)
        df_aux3["aug"]=i
        #df_aux3["model"]="resnet"

    except:
        print(f"{i} not possible")
    
    
test_scores_df=pd.concat(test_scores_df)
test_scores_df=test_scores_df.groupby("aug").mean().reset_index()

#vals=pd.read_json(f"/home/woody/iwb3/iwb3021h/THESIS_RESULTS/full_training_evonas/finetuning/{dataset}/augmentations_test_1/{dataset}/augmentation_results.json").T.reset_index().rename(columns={"index":"aug"})
#vals2=pd.read_json(f"/home/woody/iwb3/iwb3021h/THESIS_RESULTS/full_training_evonas/finetuning/{dataset}/augmentations_test_2/{dataset}/augmentation_results.json").T.reset_index().rename(columns={"index":"aug"})
vals3=pd.read_json(f"/home/woody/iwb3/iwb3021h/THESIS_RESULTS/full_training_evonas/finetuning/{dataset}/augmentations_test_resnet/{dataset}/augmentation_results.json").T.reset_index().rename(columns={"index":"aug"})
vals=pd.concat([vals3])
#vals=vals.groupby("aug").mean().reset_index()

22 not possible


In [100]:
test_scores_df.sort_values(by=["model","test_acc"])

,test_acc,aug,model
0,5.12,21,regnet
0,10.77,18,regnet
0,19.43,15,regnet
0,27.07,20,regnet
0,29.57,16,regnet
0,30.84,19,regnet
0,41.25,17,regnet
0,91.19,3,regnet
0,91.24,0,regnet
0,91.74,7,regnet


In [95]:
from scipy.stats import spearmanr
spearmanr(vals.sort_values(by="aug").val_acc.values,vals3.sort_values(by="aug").val_acc.values)

SignificanceResult(statistic=0.9265951439864484, pvalue=6.040358382311254e-10)

In [34]:
#test_scores_df=pd.concat([test_scores_df_a,test_scores_df]).groupby("aug").mean().reset_index()

In [25]:
test_scores_df.sort_values(by="test_acc",ascending=False)

,test_acc,aug,val_acc,train_acc,epoch_time
11,94.99,11,94.433334,98.415245,20.223458
12,94.63,12,94.080002,99.879807,18.494953
14,94.29,14,93.393333,97.749733,21.581148
1,94.24,1,93.646667,95.081017,28.614492
2,94.03,2,93.606667,95.922363,28.655915
13,93.71,13,92.986671,99.494743,20.952173
5,93.69,5,93.113335,87.366455,22.789521
4,93.63,4,93.426666,87.626869,23.027193
6,93.15,6,92.886665,99.396812,63.247735
9,93.02,9,92.193336,97.700768,18.509195


In [25]:
test_scores_df=pd.merge(test_scores_df, vals, on="aug")

In [26]:
test_scores_df.sort_values(by="test_acc",ascending=False)

,aug,test_acc,val_acc,train_acc,epoch_time
11,11,95.31,98.239998,97.684296,15.487517
1,1,94.79,98.009995,93.741989,32.871797
12,12,94.48,97.680000,99.791672,13.435019
14,14,93.87,96.979996,94.509216,16.848126
2,2,93.62,96.759995,91.672676,35.002472
4,4,93.61,97.399994,87.944717,25.549491
5,5,93.45,97.290001,88.044876,25.072741
10,10,93.32,96.339996,97.407852,11.305133
6,6,92.90,97.199997,99.254814,70.662297
13,13,92.71,96.659996,97.628212,16.152555


In [ ]:
ic| individual: 'devious_fossa'
ic| params: [48.0, 80, 2.249999999999999, 13, 8]

In [227]:
pearsonr(tot_dfs[["aug",metric]].groupby("aug").sum().sort_values(by="aug", ascending=True)["score_coef_norm"], test_scores_df.sort_values(by="aug")["val_acc"])

PearsonRResult(statistic=0.28123548692626665, pvalue=0.2048372823780787)

In [316]:
pd.merge(test_scores_df, tot_dfs[tot_dfs.name=="devious_fossa"], on="aug").sort_values("test_acc", ascending=False)

,test_acc,aug,val_acc,train_acc,epoch_time,name,fisher,fisher_0,score,score_coef,score_coef_norm
3,86.88,10,84.449997,98.792068,17.010701,devious_fossa,0.023535,0.021804,0.001730,0.926480,0.073520
9,86.25,16,84.409996,94.094551,21.221647,devious_fossa,0.020543,0.021804,-0.001261,1.061402,0.061402
12,86.17,19,82.579994,91.252007,23.535246,devious_fossa,0.023271,0.021804,0.001466,0.936996,0.063004
8,86.14,15,82.689995,91.538467,21.150554,devious_fossa,0.023485,0.021804,0.001680,0.928446,0.071554
0,86.13,0,84.269997,98.864189,14.790615,devious_fossa,0.021804,0.021804,0.000000,1.000000,0.000000
10,85.76,17,85.110001,88.026848,21.254751,devious_fossa,0.019654,0.021804,-0.002150,1.109414,0.109414
11,82.49,18,82.209999,93.135017,24.036993,devious_fossa,0.023492,0.021804,0.001688,0.928158,0.071842
14,82.38,21,82.379997,92.778450,26.194048,devious_fossa,0.020691,0.021804,-0.001113,1.053808,0.053808
2,81.93,9,79.809998,91.800888,19.642670,devious_fossa,0.024788,0.021804,0.002984,0.879625,0.120375
1,81.69,8,81.159996,99.801689,17.630054,devious_fossa,0.023308,0.021804,0.001503,0.935502,0.064498


In [50]:
scores=tot_dfs[["aug","score_coef"]].groupby("aug").median().sort_values("score_coef", ascending=False).reset_index()

In [51]:
merged_df=pd.merge(scores,test_scores_df, on="aug")

In [44]:
merged_df.sort_values(by="test_acc", ascending=False)

,aug,score_coef,test_acc,val_acc,train_acc,epoch_time
11,10,9.965147,86.88,84.449997,98.792068,17.010701
20,5,8.816631,86.79,85.799995,74.224762,23.364930
1,6,10.085145,86.41,83.860001,96.824921,62.911901
2,16,10.079756,86.25,84.409996,94.094551,21.221647
6,19,9.985127,86.17,82.579994,91.252007,23.535246
4,15,10.020278,86.14,82.689995,91.538467,21.150554
5,0,10.000000,86.13,84.269997,98.864189,14.790615
7,7,9.978675,86.07,84.089996,98.760017,60.961724
19,3,8.909402,86.05,83.949997,85.731171,28.971595
17,4,9.039332,85.87,85.139999,71.384216,23.384830


In [89]:
merged_df.sort_values(by="test_acc", ascending=False)

,aug,score_coef_norm,test_acc,val_acc,train_acc,epoch_time
20,11,0.232119,94.99,94.433334,98.415245,20.223458
13,12,0.343272,94.63,94.080002,99.879807,18.494953
14,14,0.341792,94.29,93.393333,97.749733,21.581148
0,1,8.240405,94.24,93.646667,95.081017,28.614492
2,2,7.479154,94.03,93.606667,95.922363,28.655915
17,13,0.267033,93.71,92.986671,99.494743,20.952173
3,5,6.109470,93.69,93.113335,87.366455,22.789521
4,4,5.828022,93.63,93.426666,87.626869,23.027193
5,6,4.805133,93.15,92.886665,99.396812,63.247735
8,9,0.478972,93.02,92.193336,97.700768,18.509195


In [52]:
merged_df.sort_values(by="test_acc", ascending=False)

,aug,score_coef,test_acc,val_acc,train_acc,epoch_time
14,13,0.987912,62.036204,63.311333,64.943913,24.226489
13,14,0.993213,61.501150,60.921093,62.664265,25.021010
7,8,1.011119,59.840984,58.570858,74.398041,19.838839
8,9,1.004996,58.880888,57.310732,66.673681,21.798818
11,12,0.998881,58.750875,57.970798,66.885017,21.632939
12,11,0.996477,58.660866,57.685770,62.024240,23.829775
5,21,1.013444,57.455746,56.020603,55.656052,28.411490
6,20,1.012827,57.385739,57.345736,53.901245,28.244505
1,18,1.019319,56.855686,57.375738,54.357973,26.540709
4,19,1.013847,54.095410,54.300430,58.433496,25.992771


In [84]:
from scipy.stats import pearsonr # Adaline jacob
pearsonr(merged_df["score_coef_norm"].values,merged_df["val_acc"].values)

PearsonRResult(statistic=-0.5547984767814877, pvalue=0.00736517217118869)

In [54]:
from scipy.stats import pearsonr # Chester jacob
pearsonr(merged_df["score_coef"].values,merged_df["val_acc"].values)

PearsonRResult(statistic=-0.5990116258949343, pvalue=0.01829090758307412)

In [45]:
from scipy.stats import pearsonr # LaMelo
pearsonr(merged_df["score_coef"].values,merged_df["test_acc"].values)

PearsonRResult(statistic=-0.2308937934371184, pvalue=0.3012176516985711)

In [90]:
from scipy.stats import pearsonr # Adaline fisher
pearsonr(merged_df["score_coef_norm"].values,merged_df["val_acc"].values)

PearsonRResult(statistic=0.42784682100367954, pvalue=0.046996268586120535)

In [80]:
tot_dfs["fisher"]=tot_dfs["fisher"].astype(float)

/home/hpc/iwb3/iwb3021h/.local/lib/python3.12/site-packages/pandas/core/dtypes/astype.py:133: ComplexWarning: Casting complex values to real discards the imaginary part
  return arr.astype(dtype, copy=True)


In [81]:
import pandas as pd
#tot_dfs=pd.read_csv(f"/home/woody/iwb3/iwb3021h/THESIS_RESULTS/5batches_fisher/fisher_{dataset}.csv", index_col=0)

#tot_dfs=pd.read_csv(f"/home/woody/iwb3/iwb3021h/THESIS_RESULTS/package2/fisher_{dataset}.csv", index_col=0)
#tot_dfs=tot_dfs[~tot_dfs.aug.isin([1,2,3,4,5,6,7])]
#tot_dfs=tot_dfs.dropna()
#batch=3
#tot_dfs=tot_dfs[tot_dfs.batch==5]

orig_dfs=tot_dfs[tot_dfs.aug==0].rename(columns={"fisher":"fisher_0"})
tot_dfs=pd.merge(tot_dfs, orig_dfs[["name","fisher_0"]], on="name")
tot_dfs["score"]=(tot_dfs["fisher"]-tot_dfs["fisher_0"]).abs()
tot_dfs["score_coef"]=tot_dfs["fisher_0"]/tot_dfs["fisher"]

#tot_dfs=tot_dfs[(tot_dfs.score_coef < 1.8)]
#tot_dfs=tot_dfs[ ((tot_dfs.score_coef > tot_dfs.score_coef.quantile(0.05)) & (tot_dfs.score_coef < tot_dfs.score_coef.quantile(0.95)))]
quantile_98 = tot_dfs['score_coef'].quantile(0.95)
quantile_02 = tot_dfs['score_coef'].quantile(0.02)
quantile_98=1.2

# Replace values in 'score_coef' column that are less than the 95th quantile with 1
#tot_dfs.loc[tot_dfs['score_coef'] < quantile_02, 'score_coef'] = 1
tot_dfs.loc[tot_dfs['score_coef'] > quantile_98, 'score_coef'] = 1

#tot_dfs=tot_dfs[ (tot_dfs.score_coef < quantile_95)]
#tot_dfs=tot_dfs[ (tot_dfs.score_coef < 1.5)]
tot_dfs["score_coef_norm"]=(tot_dfs["score_coef"]-1).abs()

tot_ranks_df=[]
metric="score_coef_norm"
for name in tot_dfs.name.unique()[:]:
    tot_ranks_df.append(tot_dfs[["name",metric,"aug"]][tot_dfs.name==name].set_index("aug")[metric].rank(ascending=False).astype(int))
tot_ranks_df=pd.concat(tot_ranks_df)

In [146]:
tot_dfs

,name,aug,fisher,fisher_0,score,score_coef,score_coef_norm
0,complex_elk,0,-134.416578,-134.416578,0.000000,1.000000,0.000000
1,complex_elk,1,-133.966246,-134.416578,0.450332,1.003362,0.003362
2,complex_elk,2,-134.362183,-134.416578,0.054395,1.000405,0.000405
3,complex_elk,3,-134.265902,-134.416578,0.150675,1.001122,0.001122
4,complex_elk,4,-134.622442,-134.416578,0.205865,0.998471,0.001529
...,...,...,...,...,...,...,...
171,versatile_bat,17,-133.867525,-134.736362,0.868837,1.006490,0.006490
172,versatile_bat,18,-134.428332,-134.736362,0.308030,1.002291,0.002291
173,versatile_bat,19,-134.466033,-134.736362,0.270329,1.002010,0.002010
174,versatile_bat,20,-134.229707,-134.736362,0.506655,1.003775,0.003775


In [42]:
import copy
#tot_dfs_1=copy.deepcopy(tot_dfs)

In [138]:
quantile_02
#tot_dfs.name.unique()[0]

0.9959239312256525

In [147]:
tot_dfs[["name",metric,"aug", "score_coef"]][tot_dfs.name==tot_dfs.name.unique()[0]].sort_values(by=metric,ascending=False)

,name,score_coef_norm,aug,score_coef
17,complex_elk,0.006718,17,1.006718
5,complex_elk,0.005515,5,0.994485
16,complex_elk,0.005416,16,1.005416
21,complex_elk,0.003371,21,1.003371
1,complex_elk,0.003362,1,1.003362
20,complex_elk,0.003266,20,1.003266
18,complex_elk,0.003096,18,1.003096
12,complex_elk,0.002728,12,1.002728
15,complex_elk,0.002700,15,1.002700
14,complex_elk,0.002620,14,1.002620


In [78]:
tot_dfs[["name",metric,"aug","score_coef","score_coef_norm","fisher"]][tot_dfs.name==tot_dfs.name.unique()[4]].set_index("aug").sort_values("score_coef")

,name,score_coef_norm,score_coef,score_coef_norm,fisher
aug,,,,,
13,purring_meerkat,0.084532,0.915468,0.084532,0.105218
12,purring_meerkat,0.055777,0.944223,0.055777,0.102013
20,purring_meerkat,0.046040,0.953960,0.046040,0.100972
18,purring_meerkat,0.041461,0.958539,0.041461,0.100490
11,purring_meerkat,0.035679,0.964321,0.035679,0.099887
8,purring_meerkat,0.023099,0.976901,0.023099,0.098601
17,purring_meerkat,0.019217,0.980783,0.019217,0.098211
14,purring_meerkat,0.015058,0.984942,0.015058,0.097796
3,purring_meerkat,0.007812,0.992188,0.007812,0.097082


In [67]:
test_scores_df.sort_values(by="test_acc",ascending=False)

,aug,test_acc,val_acc,train_acc,epoch_time
21,21,46.405,45.954999,45.108175,24.432889
18,18,43.405,43.634998,50.185297,22.342686
9,9,43.005,43.119999,52.051281,18.263696
8,8,41.660,41.455000,48.106972,16.223614
12,12,40.890,41.419999,36.880010,18.006360
13,13,38.525,38.889999,33.063903,20.687672
20,20,38.325,38.554998,31.990185,23.982806
14,14,37.775,38.529999,31.741788,21.391409
11,11,37.385,37.864999,32.343751,20.043239
1,1,31.290,31.655000,26.125801,29.595361


In [85]:
tot_dfs[["aug","score_coef"]].groupby("aug").sum().sort_values("score_coef", ascending=False)

,score_coef
aug,
21,8.743089
6,8.420235
1,8.388961
10,8.336169
19,8.334241
2,8.312965
9,8.306917
16,8.299006
17,8.263352


In [142]:
tot_dfs[["aug","score_coef"]].groupby("aug").sum().sort_values("score_coef", ascending=False)

,score_coef
aug,
2,15.860759
8,15.401335
21,14.807935
6,14.516090
16,14.223749
17,13.503060
13,13.339675
3,13.064973
15,12.518418


In [104]:
tot_dfs[["aug","score_coef"]].groupby("aug").sum().sort_values("score_coef", ascending=False)

,score_coef
aug,
17,10.130135
6,10.085145
16,10.079756
18,10.036866
15,10.020278
0,10.000000
19,9.985127
7,9.978675
21,9.969594


In [122]:
tot_dfs[["aug","score_coef_norm"]].groupby("aug").sum().sort_values("score_coef_norm", ascending=False)

,score_coef_norm
aug,
17,0.075130
16,0.067617
18,0.050790
21,0.049950
1,0.039649
3,0.039415
13,0.039309
12,0.037448
20,0.037334


In [45]:
tot_dfs[["aug","score_coef_norm"]].groupby("aug").mean().sort_values("score_coef_norm", ascending=False)

,score_coef_norm
aug,
17,0.025697
18,0.021254
16,0.020289
15,0.017102
21,0.015624
20,0.013490
19,0.013077
8,0.012317
13,0.012281


In [35]:
tot_dfs[["aug",metric]].groupby("aug").sum().sort_values(metric, ascending=False)

,score_coef_norm
aug,
17,0.119931
20,0.097952
18,0.095248
16,0.080869
8,0.080490
21,0.075291
10,0.068493
19,0.067009
14,0.066264


In [77]:
tot_dfs[["aug",metric]].groupby("aug").sum().sort_values(metric, ascending=False)

,score_coef_norm
aug,
5,1.183369
1,1.046526
2,1.027560
4,0.960668
3,0.894528
19,0.149590
14,0.141894
17,0.130135
9,0.127831


In [36]:
tot_ranks_df.reset_index().groupby("aug").std().sort_values(by=metric)

,score_coef_norm
aug,
0,0.462910
17,0.517549
8,1.281740
16,1.603567
12,2.199838
4,2.390457
9,2.672612
15,3.522884
21,3.739270


In [37]:
tot_ranks_df.reset_index().groupby("aug").sum().sort_values(by=metric)

,score_coef_norm
aug,
17,11
16,20
21,45
18,50
1,67
13,70
12,75
14,78
6,86


In [270]:
import json
import pandas as pd
from scipy.stats import spearmanr

with open(f'/home/woody/iwb3/iwb3021h/NAS_COMPETITION_RESULTS/augmentations_test/{dataset}/augmentation_results.json', 'r') as file:
    data = json.load(file)

#with open(f'/home/woody/iwb3/iwb3021h/NAS_COMPETITION_RESULTS/augmentations_test/{dataset}/augmentation_results.json', 'r') as file:
#    data = json.load(file)
    
val_scores_df=pd.DataFrame(data).T.sort_values(by="val_acc").reset_index().rename(columns={"index":"aug"})
val_scores_df.aug=val_scores_df.aug.astype(int)

aug_dfs=[]
tot_dfs=[]
for aug in range( val_scores_df.aug.max()+1):
    #if aug==5:
    df=pd.read_csv(f"zcost_proxies/results/augs/{dataset}_t{aug}.csv",index_col=0)[["fisher","grad_norm","jacob_cov","plain","snip","synflow","grasp"]].median().reset_index().set_index("index").T
    #else:
    tot_df=pd.read_csv(f"zcost_proxies/results/augs/{dataset}_t{aug}.csv",index_col=0)
    tot_df["aug"]=aug
    tot_dfs.append(tot_df)
    #    df=pd.read_csv(f"zcost_proxies/results/augs/{dataset}_t{aug}.csv",index_col=0)[["fisher","grad_norm","jacob_cov","plain","snip","synflow"]].median().reset_index().set_index("index").T
    df["aug"]=aug
    aug_dfs.append(df)
aug_dfs=pd.concat(aug_dfs)
tot_dfs=pd.concat(tot_dfs)
aug_merged=pd.merge(aug_dfs,val_scores_df[["aug","val_acc","train_acc"]],on="aug" ).sort_values(by="val_acc")

FileNotFoundError: [Errno 2] No such file or directory: 'zcost_proxies/results/augs/Adaline_t9.csv'

In [50]:
for metric in ["fisher","grad_norm","jacob_cov","plain","snip","synflow","grasp"]:
    print(metric)
    corr, _=spearmanr(aug_merged[metric],aug_merged["val_acc"])
    print(corr)

fisher
0.406060606060606
grad_norm
0.24848484848484845
jacob_cov
0.3939393939393939
plain
0.49090909090909085
snip
0.23636363636363633
synflow
-0.01818181818181818
grasp
-0.01818181818181818


In [51]:
aug_merged

,fisher,grad_norm,jacob_cov,plain,snip,synflow,grasp,aug,val_acc,train_acc
9,1.734593,147.489326,-163.766680,0.095365,752.018799,7.464972e+46,-5.589120,9,69.610001,69.953926
4,1.920861,152.362320,-163.071573,0.159077,744.056152,7.237212e+46,-12.741430,4,71.559998,70.396637
8,1.534999,139.693565,-162.608195,0.159437,696.135162,7.127571e+46,-53.899649,8,72.199997,80.855370
7,1.694661,147.189011,-161.696495,0.085732,738.489349,6.570417e+45,-8.562109,7,72.659996,78.297279
5,2.066481,160.146233,-161.740061,0.102811,795.130951,7.443510e+46,-26.605030,5,73.459999,99.389023
3,2.205899,166.330788,-160.899994,0.307613,800.787964,6.615957e+46,-57.163414,3,75.689995,86.790871
0,2.115710,160.356728,-163.839939,0.134393,776.769165,6.468904e+46,-6.987478,0,77.919998,100.000000
6,1.780227,150.153137,-162.535070,0.149173,766.213684,7.542605e+46,-30.239204,6,79.259995,99.549278
1,2.081749,164.913582,-162.665356,0.214968,792.663269,7.680437e+46,-5.399981,1,80.439995,98.866188
2,1.760488,143.141624,-160.342060,0.193770,733.858002,6.236257e+46,-13.618480,2,82.369995,86.083740


In [327]:
tot_dfs[["name",metric,"aug"]][~tot_dfs["aug"].isin([])].sort_values(by=["name",metric], ascending=[True,False])

,name,score_coef,aug
254,astonishing_hedgehog,1.383329,1
260,astonishing_hedgehog,1.356938,7
265,astonishing_hedgehog,1.326069,12
274,astonishing_hedgehog,1.247565,21
261,astonishing_hedgehog,1.240422,8
...,...,...,...
38,xanthic_ara,1.067653,15
40,xanthic_ara,1.066075,17
34,xanthic_ara,1.045001,11
43,xanthic_ara,1.039397,20


In [133]:
rank_df=[]#

metric="score_coef"
rank_df=tot_dfs[["name",metric,"aug"]][~tot_dfs["aug"].isin([4,0,7])].sort_values(by=["name",metric], ascending=[True,True]).drop_duplicates(subset=["name"], keep="first")["aug"].value_counts()        
print(rank_df)
rank_df_ranking=rank_df.reset_index().groupby("aug").sum().sort_values(by="count")

aug
10    5
5     3
Name: count, dtype: int64


In [129]:
tot_ranks_df=[]
metric="fisher"
for name in tot_dfs.name.unique()[:]:
    tot_ranks_df.append(tot_dfs[["name",metric,"aug"]][tot_dfs.name==name].set_index("aug")[metric].rank().astype(int))
tot_ranks_df=pd.concat(tot_ranks_df)

In [14]:
tot_ranks_df.reset_index().groupby("aug").std().sort_values(by=metric)

,fisher
aug,
9,1.327265
4,1.411469
7,1.531173
0,1.643913
5,1.749752
1,1.764156
3,1.772811
8,1.793867
6,1.981651


In [15]:
tot_ranks_df.reset_index().groupby("aug").sum().sort_values(by=metric)

,fisher
aug,
9,122
4,137
8,146
7,166
2,317
3,330
5,357
1,375
6,377


In [53]:
tot_ranks_df.reset_index().groupby("aug").sum().sort_values(by="fisher").head(2).index[1]

KeyError: 'fisher'

In [86]:
rank_df_counts=pd.merge(aug_merged[["aug","val_acc","train_acc"]],rank_df_ranking.reset_index(),on="aug",how="inner")
rank_df_counts["dataset"]=dataset


In [87]:
aug_merged

,fisher,grad_norm,jacob_cov,plain,snip,synflow,grasp,aug,val_acc,train_acc
9,1.734593,147.489326,-163.766680,0.095365,752.018799,7.464972e+46,-5.589120,9,69.610001,69.953926
4,1.920861,152.362320,-163.071573,0.159077,744.056152,7.237212e+46,-12.741430,4,71.559998,70.396637
8,1.534999,139.693565,-162.608195,0.159437,696.135162,7.127571e+46,-53.899649,8,72.199997,80.855370
7,1.694661,147.189011,-161.696495,0.085732,738.489349,6.570417e+45,-8.562109,7,72.659996,78.297279
5,2.066481,160.146233,-161.740061,0.102811,795.130951,7.443510e+46,-26.605030,5,73.459999,99.389023
3,2.205899,166.330788,-160.899994,0.307613,800.787964,6.615957e+46,-57.163414,3,75.689995,86.790871
0,2.115710,160.356728,-163.839939,0.134393,776.769165,6.468904e+46,-6.987478,0,77.919998,100.000000
6,1.780227,150.153137,-162.535070,0.149173,766.213684,7.542605e+46,-30.239204,6,79.259995,99.549278
1,2.081749,164.913582,-162.665356,0.214968,792.663269,7.680437e+46,-5.399981,1,80.439995,98.866188
2,1.760488,143.141624,-160.342060,0.193770,733.858002,6.236257e+46,-13.618480,2,82.369995,86.083740


In [88]:
rank_df_counts

,aug,val_acc,train_acc,count,dataset
0,9,69.610001,69.953926,1,LaMelo
1,4,71.559998,70.396637,5,LaMelo
2,8,72.199997,80.855370,3,LaMelo
3,7,72.659996,78.297279,1,LaMelo
4,5,73.459999,99.389023,1,LaMelo
5,3,75.689995,86.790871,1,LaMelo
6,6,79.259995,99.549278,2,LaMelo
7,1,80.439995,98.866188,1,LaMelo
8,2,82.369995,86.083740,5,LaMelo


In [37]:
rank_df_counts.to_csv(f"zcost_proxies/results/augs/rank_{dataset}.csv")

In [38]:
spearmanr(rank_df_counts["val_acc"],rank_df_counts["count"])

SignificanceResult(statistic=0.03465834966066909, pvalue=0.9294652277392534)

In [178]:
rank_df_ranking.index.values

array([ 4,  2,  3,  6,  8,  0,  9,  1,  7, 10,  5])

In [179]:
aug_merged.aug.values

array([4, 3, 1, 2, 7, 5, 0, 8, 6])

In [138]:
rank_df_ranking.index.values

array([2, 3, 6, 7, 4, 1, 8, 0, 5, 9])

In [139]:
aug_merged.aug.values

array([ 4,  3,  0,  9,  5,  7,  2,  6,  1,  8, 10])